# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
from sys import getsizeof
import cv2
import matplotlib.pyplot as plt
% matplotlib inline

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
project_folder='Project_data'

train_doc = np.random.permutation(open(project_folder + '/' + 'train.csv').readlines())
val_doc = np.random.permutation(open(project_folder + '/' + 'val.csv').readlines())

num_channels = 3

In [4]:
def set_hyper_parameters(batchSize, numFrames, numRows, numCols, numEpochs):
    global batch_size, num_frames, num_rows, num_cols, num_epochs, input_shape
    batch_size = batchSize
    num_frames = numFrames
    num_rows = numRows
    num_cols = numCols
    num_epochs =numEpochs
    input_shape=(num_frames,num_rows,num_cols,num_channels)
    print("batch_size=", batch_size, ",num_frames=", num_frames, ",num_rows=", num_rows, ",num_cols=", num_cols, ",num_epochs=", num_epochs)

In [5]:
set_hyper_parameters(batchSize=10, numFrames=30, numRows=60, numCols=60, numEpochs=2)

batch_size= 10 ,num_frames= 30 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 2


In [6]:
input_shape

(30, 60, 60, 3)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [7]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = img_idx = np.round(np.linspace(0,29,num_frames)).astype(int)
    while True:
        num_batches = len(folder_list)//batch_size
        t = np.random.permutation(folder_list)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,num_frames,num_rows,num_cols,num_channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # Cropping non symmetric frames
                    if image.shape[0] != image.shape[1]:
                        image=image[0:120,20:140]
                        
                    image = imresize(image,(num_rows, num_cols)) #resize the image
                    #image = image/255 #normalize the image
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        # batch_size = 20
        # folder_list = 663
        # num_batches = 33
        # last batch size = 663 - 20* 33 = 3
        
        if (len(folder_list) > num_batches * batch_size):
            batch_size = len(folder_list) - num_batches * batch_size
            batch_data = np.zeros((batch_size,num_frames,num_rows,num_cols,num_channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    # Cropping non symmetric frames
                    if image.shape[0] != image.shape[1]:
                        image=image[0:120,20:140]
                        
                    image = imresize(image,(num_rows, num_cols)) #resize the image
                    #image = image/255 #normalize the image
                                        
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = project_folder + '/' + 'train'
val_path = project_folder + '/' + 'val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
#num_epochs =  20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 2


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Conv3D, MaxPooling3D, Convolution2D, MaxPooling2D 
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications import ResNet50

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
def set_train_validation_steps():
    global steps_per_epoch, validation_steps
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1
    print("steps_per_epoch=", steps_per_epoch, "validation_steps=",validation_steps)

## Model 1: Experimenting with a large batch_size=60

In [10]:
# #write your model here
# def get_conv3d_model():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2,2,2)))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#     model.add(Conv3D(128, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))

#     model.add(Dense(128, activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))

#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [16]:
# # Set Hyperparameters for the model by calling the set_hyper_parameters function
# set_hyper_parameters(batchSize=60, numFrames=30, numRows=120, numCols=120, numEpochs=2)

batch_size= 60 ,num_frames= 30 ,num_rows= 120 ,num_cols= 120 ,num_epochs= 2


In [17]:
# # Deriving the steps_per_epoch and validation_steps which are used by fit_generator to decide the number of next() calls it need to make
# set_train_validation_steps()

steps_per_epoch= 12 validation_steps= 2


In [21]:
# # Calling the base model
# model1 = get_conv3d_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_6 (Activation)    (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 15, 60, 60, 32)    128       
__________

In [22]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

#### Commenting out this code section since this was used to identify the optimal batch size

In [23]:
# model1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 60
Source path =  Project_data/train Epoch 1/2
; batch size = 60


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


ResourceExhaustedError: OOM when allocating tensor with shape[60,16,30,120,120] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_1/Adam/gradients/max_pooling3d_5/MaxPool3D_grad/MaxPool3DGrad = MaxPool3DGrad[T=DT_FLOAT, TInput=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/batch_normalization_7/cond/Merge_grad/cond_grad"], data_format="NDHWC", ksize=[1, 2, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](batch_normalization_7/cond/Merge, max_pooling3d_5/MaxPool3D, training_1/Adam/gradients/conv3d_6/convolution_grad/Conv3DBackpropInputV2)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


## Model 2: Reducing the batch size to 40

In [24]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=30, numRows=120, numCols=120, numEpochs=2)

batch_size= 40 ,num_frames= 30 ,num_rows= 120 ,num_cols= 120 ,num_epochs= 2


In [25]:
# # Deriving the steps_per_epoch and validation_steps which are used by fit_generator to decide the number of next() calls it need to make
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [26]:
# # Calling the base model
# model2 = get_conv3d_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_10 (Activation)   (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 15, 60, 60, 32)    128       
__________

In [27]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [28]:
# model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/2


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 149s 9s/step - loss: 1.4106 - categorical_accuracy: 0.5174 - val_loss: 3.8075 - val_categorical_accuracy: 0.3400

Epoch 00001: saving model to model_init_2019-12-2217_21_14.883121/model-00001-1.43628-0.50679-3.80752-0.34000.h5
Epoch 2/2
17/17 [==============================] - 45s 3s/step - loss: 0.9017 - categorical_accuracy: 0.6471 - val_loss: 1.7861 - val_categorical_accuracy: 0.4833

Epoch 00002: saving model to model_init_2019-12-2217_21_14.883121/model-00002-0.90173-0.64706-1.78607-0.48333.h5


40 seems to be a viable batch size

### Model 3: The strategy here is to start small and then increase the complexity of the model if required

In [41]:
# #write your model here
# def get_conv3d_model_a():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling3D(pool_size=(2,2,2)))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))

#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [42]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [43]:
# # Deriving the steps_per_epoch and validation_steps which are used by fit_generator to decide the number of next() calls it need to make
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [44]:
# model3 = get_conv3d_model_a()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_22 (Conv3D)           (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_22 (Activation)   (None, 16, 60, 60, 16)    0         
_________________________________________________________________
max_pooling3d_22 (MaxPooling (None, 8, 30, 30, 16)     0         
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 8, 30, 30, 32)     13856     
_________________________________________________________________
activation_23 (Activation)   (None, 8, 30, 30, 32)     0         
_________________________________________________________________
max_pooling3d_23 (MaxPooling (None, 4, 15, 15, 32)     0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 4, 15, 15, 64)     55360     
__________

In [45]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [47]:
# model3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 18s 1s/step - loss: 0.8033 - categorical_accuracy: 0.6747 - val_loss: 0.9505 - val_categorical_accuracy: 0.6167

Epoch 00001: saving model to model_init_2019-12-2217_21_14.883121/model-00001-0.80330-0.67474-0.95048-0.61667.h5
Epoch 2/50
17/17 [==============================] - 16s 937ms/step - loss: 0.7521 - categorical_accuracy: 0.6920 - val_loss: 0.7844 - val_categorical_accuracy: 0.6500

Epoch 00002: saving model to model_init_2019-12-2217_21_14.883121/model-00002-0.75214-0.69204-0.78435-0.65000.h5
Epoch 3/50
17/17 [==============================] - 16s 956ms/step - loss: 0.7167 - categorical_accuracy: 0.7266 - val_loss: 1.0998 - val_categorical_accuracy: 0.5833

Epoch 00003: saving model to model_init_2019-12-2217_21_14.883121/model-00003-0.71671-0.72664-1.09982-0.58333.h5
Epoch 4/50
17/17 [==============================] - 16s 934ms/step - loss: 0.5715 - categorical_accuracy: 0.7543 - val_loss: 1.0872 - val_categorical_accuracy: 0.5833

Epo


Epoch 00027: saving model to model_init_2019-12-2217_21_14.883121/model-00027-0.11740-0.97232-0.92296-0.73333.h5
Epoch 28/50
17/17 [==============================] - 15s 891ms/step - loss: 0.1168 - categorical_accuracy: 0.9689 - val_loss: 0.8552 - val_categorical_accuracy: 0.7167

Epoch 00028: saving model to model_init_2019-12-2217_21_14.883121/model-00028-0.11679-0.96886-0.85520-0.71667.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.
Epoch 29/50
17/17 [==============================] - 16s 962ms/step - loss: 0.1348 - categorical_accuracy: 0.9689 - val_loss: 0.9686 - val_categorical_accuracy: 0.7333

Epoch 00029: saving model to model_init_2019-12-2217_21_14.883121/model-00029-0.13476-0.96886-0.96864-0.73333.h5
Epoch 30/50
17/17 [==============================] - 16s 921ms/step - loss: 0.1211 - categorical_accuracy: 0.9654 - val_loss: 0.7733 - val_categorical_accuracy: 0.7167

Epoch 00030: saving model to model_init_2019-12-2217_21_14.883121/model-

Epoch 24/50
17/17 [==============================] - 16s 924ms/step - loss: 0.1505 - categorical_accuracy: 0.9585 - val_loss: 0.5889 - val_categorical_accuracy: 0.7833

Epoch 00024: saving model to model_init_2019-12-2217_21_14.883121/model-00024-0.15052-0.95848-0.58886-0.78333.h5


The model seems to be clearly overfitting with training accuracy of 0.9585 and validation accuracy of 0.7844

## Model 4 : Introducing drop outs in CNN layers  and FC layer to reduce overfitting


In [13]:
# def get_conv3d_model_b():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling3D(pool_size=(2,2,2)))
#     model.add(Dropout(0.25))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(Dropout(0.25))

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(Dropout(0.25))

#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.25))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))


#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [14]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [15]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [16]:
# ## Calling the model function
# model4 = get_conv3d_model_b()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 60, 60, 16)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 8, 30, 30, 16)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 30, 30, 16)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 8, 30, 30, 32)     13856     
_________________________________________________________________
activation_2 (Activation)    (None, 8, 30, 30, 32)     0         
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 4, 15, 15, 32)     0         
__________

In [17]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [19]:
# model4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 19s 1s/step - loss: 0.5440 - categorical_accuracy: 0.7993 - val_loss: 0.6375 - val_categorical_accuracy: 0.7167

Epoch 00001: saving model to model_init_2019-12-2218_27_33.746320/model-00001-0.54399-0.79931-0.63745-0.71667.h5
Epoch 2/50
17/17 [==============================] - 16s 950ms/step - loss: 0.4666 - categorical_accuracy: 0.8097 - val_loss: 0.6566 - val_categorical_accuracy: 0.6833

Epoch 00002: saving model to model_init_2019-12-2218_27_33.746320/model-00002-0.46664-0.80969-0.65662-0.68333.h5
Epoch 3/50
17/17 [==============================] - 16s 919ms/step - loss: 0.5124 - categorical_accuracy: 0.7924 - val_loss: 0.6064 - val_categorical_accuracy: 0.7667

Epoch 00003: saving model to model_init_2019-12-2218_27_33.746320/model-00003-0.51239-0.79239-0.60645-0.76667.h5
Epoch 4/50
17/17 [==============================] - 16s 915ms/step - loss: 0.5055 - categorical_accuracy: 0.8131 - val_loss: 0.6395 - val_categorical_accuracy: 0.7500

Epo


Epoch 00027: saving model to model_init_2019-12-2218_27_33.746320/model-00027-0.44142-0.85467-0.61722-0.75000.h5
Epoch 28/50
17/17 [==============================] - 16s 929ms/step - loss: 0.4638 - categorical_accuracy: 0.8339 - val_loss: 0.6497 - val_categorical_accuracy: 0.7000

Epoch 00028: saving model to model_init_2019-12-2218_27_33.746320/model-00028-0.46385-0.83391-0.64965-0.70000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 9.536743617033494e-10.
Epoch 29/50
17/17 [==============================] - 17s 971ms/step - loss: 0.4980 - categorical_accuracy: 0.7958 - val_loss: 0.7417 - val_categorical_accuracy: 0.7500

Epoch 00029: saving model to model_init_2019-12-2218_27_33.746320/model-00029-0.49796-0.79585-0.74165-0.75000.h5
Epoch 30/50
17/17 [==============================] - 15s 907ms/step - loss: 0.5134 - categorical_accuracy: 0.7855 - val_loss: 0.6108 - val_categorical_accuracy: 0.6667

Epoch 00030: saving model to model_init_2019-12-2218_27_33.746320/model-

Epoch 24/50
17/17 [==============================] - 16s 919ms/step - loss: 0.4724 - categorical_accuracy: 0.8201 - val_loss: 0.5447 - val_categorical_accuracy: 0.7833

Epoch 00024: saving model to model_init_2019-12-2218_27_33.746320/model-00024-0.47244-0.82007-0.54474-0.78333.h5


## Model 5 : Remove dropouts from conv layers, retain dropouts in FC, use BN

In [28]:
# def get_conv3d_model_c():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2,2,2)))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   

#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.25))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))


#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [29]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [30]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [31]:
# ## Calling the model function
# model5 = get_conv3d_model_c()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_10 (Activation)   (None, 16, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 16, 60, 60, 16)    64        
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 8, 30, 30, 16)     0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 8, 30, 30, 32)     13856     
_________________________________________________________________
activation_11 (Activation)   (None, 8, 30, 30, 32)     0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 8, 30, 30, 32)     128       
__________

In [32]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [33]:
# model5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train Epoch 1/50
 Project_data/val ; batch size = 40
; batch size = 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/17 [==>...........................] - ETA: 46s - loss: 5.8230 - categorical_accuracy: 0.1750 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 40s 2s/step - loss: 6.8811 - categorical_accuracy: 0.3191 - val_loss: 9.1456 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2019-12-2218_27_33.746320/model-00001-6.93266-0.31523-9.14557-0.32000.h5
Epoch 2/50
17/17 [==============================] - 21s 1s/step - loss: 5.4412 - categorical_accuracy: 0.4476 - val_loss: 7.0022 - val_categorical_accuracy: 0.4167

Epoch 00002: saving model to model_init_2019-12-2218_27_33.746320/model-00002-5.44119-0.44757-7.00216-0.41667.h5
Epoch 3/50
17/17 [==============================] - 20s 1s/step - loss: 4.6952 - categorical_accuracy: 0.4664 - val_loss: 9.1679 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2019-12-2218_27_33.746320/model-00003-4.72082-0.46866-9.16790-0.20000.h5
Epoch 4/50
17/17 [==============================] - 17s 1s/step - loss: 2.3112 - categorical_accuracy: 0.4458 - val_loss: 2.4679 - val_categorical_accuracy: 0.3833

Epoch 00004:

17/17 [==============================] - 17s 977ms/step - loss: 0.0862 - categorical_accuracy: 0.9827 - val_loss: 0.8910 - val_categorical_accuracy: 0.7167

Epoch 00029: saving model to model_init_2019-12-2218_27_33.746320/model-00029-0.08624-0.98270-0.89104-0.71667.h5
Epoch 30/50
17/17 [==============================] - 16s 935ms/step - loss: 0.0728 - categorical_accuracy: 0.9896 - val_loss: 0.4333 - val_categorical_accuracy: 0.8167

Epoch 00030: saving model to model_init_2019-12-2218_27_33.746320/model-00030-0.07278-0.98962-0.43332-0.81667.h5

Epoch 00030: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 31/50
17/17 [==============================] - 15s 910ms/step - loss: 0.0887 - categorical_accuracy: 0.9758 - val_loss: 0.8118 - val_categorical_accuracy: 0.7167

Epoch 00031: saving model to model_init_2019-12-2218_27_33.746320/model-00031-0.08869-0.97578-0.81185-0.71667.h5
Epoch 32/50
17/17 [==============================] - 17s 999ms/step - loss: 0.0907 -

Epoch 23/50
17/17 [==============================] - 16s 918ms/step - loss: 0.1172 - categorical_accuracy: 0.9619 - val_loss: 0.4453 - val_categorical_accuracy: 0.8500

Epoch 00023: saving model to model_init_2019-12-2218_27_33.746320/model-00023-0.11722-0.96194-0.44531-0.85000.h5

### Model 6: Use dropouts after conv and FC layers, use BN

In [34]:
# def get_conv3d_model_d():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2,2,2)))
#     model.add(Dropout(0.25))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(Dropout(0.25))
   

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(Dropout(0.25))
   

#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.25))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))


#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [35]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [36]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [37]:
# ## Calling the model function
# model6 = get_conv3d_model_d()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_13 (Activation)   (None, 16, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 16, 60, 60, 16)    64        
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 8, 30, 30, 16)     0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 8, 30, 30, 16)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 8, 30, 30, 32)     13856     
_________________________________________________________________
activation_14 (Activation)   (None, 8, 30, 30, 32)     0         
__________

In [38]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [39]:
# model6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/17 [==>...........................] - ETA: 41s - loss: 5.3121 - categorical_accuracy: 0.1875 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 39s 2s/step - loss: 9.5619 - categorical_accuracy: 0.3107 - val_loss: 11.1289 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2019-12-2218_27_33.746320/model-00001-9.57279-0.30769-11.12894-0.28000.h5
Epoch 2/50
17/17 [==============================] - 20s 1s/step - loss: 10.1282 - categorical_accuracy: 0.3529 - val_loss: 10.2308 - val_categorical_accuracy: 0.3500

Epoch 00002: saving model to model_init_2019-12-2218_27_33.746320/model-00002-10.12823-0.35294-10.23085-0.35000.h5
Epoch 3/50
17/17 [==============================] - 20s 1s/step - loss: 10.4342 - categorical_accuracy: 0.3345 - val_loss: 10.4223 - val_categorical_accuracy: 0.3500

Epoch 00003: saving model to model_init_2019-12-2218_27_33.746320/model-00003-10.42121-0.33515-10.42233-0.35000.h5
Epoch 4/50
17/17 [==============================] - 18s 1s/step - loss: 9.9120 - categorical_accuracy: 0.3808 - val_loss: 10.8598 - val_categorical_accuracy: 0.3167

E


Epoch 00028: saving model to model_init_2019-12-2218_27_33.746320/model-00028-0.82279-0.71972-1.83140-0.68333.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 29/50
17/17 [==============================] - 16s 955ms/step - loss: 0.7817 - categorical_accuracy: 0.7370 - val_loss: 1.6874 - val_categorical_accuracy: 0.5500

Epoch 00029: saving model to model_init_2019-12-2218_27_33.746320/model-00029-0.78171-0.73702-1.68741-0.55000.h5
Epoch 30/50
17/17 [==============================] - 15s 884ms/step - loss: 0.8143 - categorical_accuracy: 0.7024 - val_loss: 1.5288 - val_categorical_accuracy: 0.6500

Epoch 00030: saving model to model_init_2019-12-2218_27_33.746320/model-00030-0.81428-0.70242-1.52884-0.65000.h5
Epoch 31/50
17/17 [==============================] - 16s 921ms/step - loss: 0.7328 - categorical_accuracy: 0.7578 - val_loss: 1.7412 - val_categorical_accuracy: 0.5333

Epoch 00031: saving model to model_init_2019-12-2218_27_33.746320/model-

### Model 7: Add a new convolutional layer to the network

In [40]:
# def get_conv3d_model_e():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(Conv3D(16, kernel_size=(3,3,3)))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2,2,2)))
#     model.add(Dropout(0.25))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(Dropout(0.25))
   

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(Dropout(0.25))
   

#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.25))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))


#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [41]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [42]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [43]:
# ## Calling the model function
# model7 = get_conv3d_model_e()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_16 (Activation)   (None, 16, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 16, 60, 60, 16)    64        
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 14, 58, 58, 16)    6928      
_________________________________________________________________
activation_17 (Activation)   (None, 14, 58, 58, 16)    0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 14, 58, 58, 16)    64        
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 7, 29, 29, 16)     0         
__________

In [44]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [46]:
# model7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 19s 1s/step - loss: 0.6152 - categorical_accuracy: 0.7578 - val_loss: 0.8330 - val_categorical_accuracy: 0.6500

Epoch 00001: saving model to model_init_2019-12-2218_27_33.746320/model-00001-0.61520-0.75779-0.83304-0.65000.h5
Epoch 2/50
17/17 [==============================] - 17s 981ms/step - loss: 0.5237 - categorical_accuracy: 0.7993 - val_loss: 0.9896 - val_categorical_accuracy: 0.6667

Epoch 00002: saving model to model_init_2019-12-2218_27_33.746320/model-00002-0.52370-0.79931-0.98963-0.66667.h5
Epoch 3/50
17/17 [==============================] - 16s 922ms/step - loss: 0.5137 - categorical_accuracy: 0.7958 - val_loss: 1.2281 - val_categorical_accuracy: 0.5167

Epoch 00003: saving model to model_init_2019-12-2218_27_33.746320/model-00003-0.51375-0.79585-1.22810-0.51667.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 4/50
17/17 [==============================] - 16s 962ms/step - loss: 0.4941 - catego

17/17 [==============================] - 16s 964ms/step - loss: 0.2541 - categorical_accuracy: 0.9170 - val_loss: 1.1609 - val_categorical_accuracy: 0.6500

Epoch 00027: saving model to model_init_2019-12-2218_27_33.746320/model-00027-0.25412-0.91696-1.16089-0.65000.h5
Epoch 28/50
17/17 [==============================] - 16s 965ms/step - loss: 0.2633 - categorical_accuracy: 0.9066 - val_loss: 0.7144 - val_categorical_accuracy: 0.7167

Epoch 00028: saving model to model_init_2019-12-2218_27_33.746320/model-00028-0.26326-0.90657-0.71437-0.71667.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.
Epoch 29/50
17/17 [==============================] - 17s 1000ms/step - loss: 0.3440 - categorical_accuracy: 0.8720 - val_loss: 0.9665 - val_categorical_accuracy: 0.7333

Epoch 00029: saving model to model_init_2019-12-2218_27_33.746320/model-00029-0.34397-0.87197-0.96655-0.73333.h5
Epoch 30/50
17/17 [==============================] - 15s 888ms/step - loss: 0.2746 

Epoch 31/50
17/17 [==============================] - 16s 944ms/step - loss: 0.3339 - categorical_accuracy: 0.8720 - val_loss: 0.7943 - val_categorical_accuracy: 0.7667

Epoch 00031: saving model to model_init_2019-12-2218_27_33.746320/model-00031-0.33387-0.87197-0.79427-0.76667.h5

#### Model 8: Increase dropout ratio at the Dense layer 

In [18]:
# def get_conv3d_model_f():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2,2,2)))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   

#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.50))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))


#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [19]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [20]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [21]:
# model8 = get_conv3d_model_f()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 60, 60, 16)    64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 8, 30, 30, 16)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 8, 30, 30, 32)     13856     
_________________________________________________________________
activation_2 (Activation)    (None, 8, 30, 30, 32)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 30, 30, 32)     128       
__________

In [22]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [23]:
# model8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 135s 8s/step - loss: 6.7103 - categorical_accuracy: 0.3300 - val_loss: 7.5924 - val_categorical_accuracy: 0.4300

Epoch 00001: saving model to model_init_2019-12-2222_33_48.524896/model-00001-6.77673-0.32428-7.59240-0.43000.h5
Epoch 2/50
17/17 [==============================] - 24s 1s/step - loss: 2.7805 - categorical_accuracy: 0.4910 - val_loss: 2.5668 - val_categorical_accuracy: 0.4500

Epoch 00002: saving model to model_init_2019-12-2222_33_48.524896/model-00002-2.78046-0.49105-2.56681-0.45000.h5
Epoch 3/50
17/17 [==============================] - 22s 1s/step - loss: 1.6449 - categorical_accuracy: 0.5192 - val_loss: 1.7703 - val_categorical_accuracy: 0.3667

Epoch 00003: saving model to model_init_2019-12-2222_33_48.524896/model-00003-1.66571-0.51771-1.77034-0.36667.h5
Epoch 4/50
17/17 [==============================] - 18s 1s/step - loss: 1.2436 - categorical_accuracy: 0.5387 - val_loss: 1.0854 - val_categorical_accuracy: 0.5833

Epoch 00004

Epoch 20/50
17/17 [==============================] - 17s 978ms/step - loss: 0.3701 - categorical_accuracy: 0.8651 - val_loss: 0.5243 - val_categorical_accuracy: 0.8333

Epoch 00020: saving model to model_init_2019-12-2222_33_48.524896/model-00020-0.37013-0.86505-0.52433-0.83333.h5

## Model 9: Continuation of model 7 by increasing drop out ratio at Dense Layer

In [24]:
# def get_conv3d_model_g():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(Conv3D(16, kernel_size=(3,3,3)))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2,2,2)))
#     model.add(Dropout(0.25))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(Dropout(0.25))
   

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#     model.add(Dropout(0.25))
   

#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.50))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))


#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [25]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [26]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [28]:
# model9 = get_conv3d_model_g()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_7 (Activation)    (None, 16, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 16, 60, 60, 16)    64        
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 14, 58, 58, 16)    6928      
_________________________________________________________________
activation_8 (Activation)    (None, 14, 58, 58, 16)    0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 14, 58, 58, 16)    64        
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 7, 29, 29, 16)     0         
__________

In [29]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [30]:
# model9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = Source path =  40
Project_data/train ; batch size = 40
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 42s 2s/step - loss: 4.7456 - categorical_accuracy: 0.3177 - val_loss: 2.4397 - val_categorical_accuracy: 0.3400

Epoch 00001: saving model to model_init_2019-12-2222_33_48.524896/model-00001-4.83910-0.31071-2.43972-0.34000.h5
Epoch 2/50
17/17 [==============================] - 20s 1s/step - loss: 1.7170 - categorical_accuracy: 0.4143 - val_loss: 1.5125 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2019-12-2222_33_48.524896/model-00002-1.71699-0.41432-1.51249-0.40000.h5
Epoch 3/50
17/17 [==============================] - 21s 1s/step - loss: 1.2275 - categorical_accuracy: 0.5243 - val_loss: 1.4335 - val_categorical_accuracy: 0.5833

Epoch 00003: saving model to model_init_2019-12-2222_33_48.524896/model-00003-1.23874-0.52044-1.43353-0.58333.h5
Epoch 4/50
17/17 [==============================] - 18s 1s/step - loss: 1.1348 - categorical_accuracy: 0.5449 - val_loss: 1.2171 - val_categorical_accuracy: 0.5833

Epoch 00004:


Epoch 00028: saving model to model_init_2019-12-2222_33_48.524896/model-00028-0.30333-0.88927-0.96078-0.63333.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
Epoch 29/50
17/17 [==============================] - 17s 996ms/step - loss: 0.2536 - categorical_accuracy: 0.9066 - val_loss: 0.7171 - val_categorical_accuracy: 0.7000

Epoch 00029: saving model to model_init_2019-12-2222_33_48.524896/model-00029-0.25357-0.90657-0.71714-0.70000.h5
Epoch 30/50
17/17 [==============================] - 16s 915ms/step - loss: 0.2241 - categorical_accuracy: 0.9170 - val_loss: 0.7684 - val_categorical_accuracy: 0.6667

Epoch 00030: saving model to model_init_2019-12-2222_33_48.524896/model-00030-0.22410-0.91696-0.76840-0.66667.h5

Epoch 00030: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 31/50
17/17 [==============================] - 16s 942ms/step - loss: 0.2817 - categorical_accuracy: 0.8997 - val_loss: 0.9183 - val_categorical_accuracy:

Epoch 15/50
17/17 [==============================] - 17s 982ms/step - loss: 0.4898 - categorical_accuracy: 0.8028 - val_loss: 0.7186 - val_categorical_accuracy: 0.7833

Epoch 00015: saving model to model_init_2019-12-2222_33_48.524896/model-00015-0.48977-0.80277-0.71860-0.78333.h5

## Model 10: Enhancements to model 8 by adding an additional CNN layer with 128 filters

In [38]:
def get_conv3d_model_h():

# Define model
    model = Sequential()

    model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2,2,2)))

    model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   

    model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   
    model.add(Conv3D(128, kernel_size=(3,3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   
    #Flatten Layers
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.50))

    #softmax layer
    model.add(Dense(5, activation='softmax'))


    optimiser = optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())
    return model

In [39]:
# Set Hyperparameters for the model
set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [40]:
set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [41]:
model10 = get_conv3d_model_h()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_21 (Activation)   (None, 16, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 16, 60, 60, 16)    64        
_________________________________________________________________
max_pooling3d_18 (MaxPooling (None, 8, 30, 30, 16)     0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 8, 30, 30, 32)     13856     
_________________________________________________________________
activation_22 (Activation)   (None, 8, 30, 30, 32)     0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 8, 30, 30, 32)     128       
__________

In [42]:
# create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [43]:
model10.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/val Project_data/train ; batch size = 40
 Epoch 1/50; batch size = 40



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 1/17 [>.............................] - ETA: 1:32 - loss: 3.6034 - categorical_accuracy: 0.3000

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 39s 2s/step - loss: 2.1950 - categorical_accuracy: 0.3843 - val_loss: 1.2634 - val_categorical_accuracy: 0.5300

Epoch 00001: saving model to model_init_2019-12-2222_33_48.524896/model-00001-2.21368-0.38311-1.26340-0.53000.h5
Epoch 2/50
17/17 [==============================] - 21s 1s/step - loss: 1.3038 - categorical_accuracy: 0.4859 - val_loss: 1.2827 - val_categorical_accuracy: 0.5500

Epoch 00002: saving model to model_init_2019-12-2222_33_48.524896/model-00002-1.30378-0.48593-1.28269-0.55000.h5
Epoch 3/50
17/17 [==============================] - 21s 1s/step - loss: 1.0019 - categorical_accuracy: 0.5423 - val_loss: 0.8755 - val_categorical_accuracy: 0.6333

Epoch 00003: saving model to model_init_2019-12-2222_33_48.524896/model-00003-1.01468-0.53678-0.87553-0.63333.h5
Epoch 4/50
17/17 [==============================] - 17s 1s/step - loss: 0.9793 - categorical_accuracy: 0.5975 - val_loss: 1.0946 - val_categorical_accuracy: 0.6167

Epoch 00004:

17/17 [==============================] - 16s 956ms/step - loss: 0.2950 - categorical_accuracy: 0.8754 - val_loss: 0.4747 - val_categorical_accuracy: 0.8500

Epoch 00028: saving model to model_init_2019-12-2222_33_48.524896/model-00028-0.29502-0.87543-0.47467-0.85000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
Epoch 29/50
17/17 [==============================] - 16s 944ms/step - loss: 0.2686 - categorical_accuracy: 0.9031 - val_loss: 0.3631 - val_categorical_accuracy: 0.8833

Epoch 00029: saving model to model_init_2019-12-2222_33_48.524896/model-00029-0.26855-0.90311-0.36314-0.88333.h5
Epoch 30/50
17/17 [==============================] - 16s 951ms/step - loss: 0.2390 - categorical_accuracy: 0.9412 - val_loss: 0.7397 - val_categorical_accuracy: 0.7167

Epoch 00030: saving model to model_init_2019-12-2222_33_48.524896/model-00030-0.23896-0.94118-0.73965-0.71667.h5
Epoch 31/50
17/17 [==============================] - 16s 948ms/step - loss: 0.2478 -

Epoch 29/50
17/17 [==============================] - 16s 944ms/step - loss: 0.2686 - categorical_accuracy: 0.9031 - val_loss: 0.3631 - val_categorical_accuracy: 0.8833

Epoch 00029: saving model to model_init_2019-12-2222_33_48.524896/model-00029-0.26855-0.90311-0.36314-0.88333.h5

## Model 11: Increasing the no of neurons in the dense layer

In [44]:
# def get_conv3d_model_i():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2,2,2)))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   
#     model.add(Conv3D(128, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   
#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(512, activation='relu'))
#     model.add(Dropout(0.50))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))


#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [45]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=60, numCols=60, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 60 ,num_cols= 60 ,num_epochs= 50


In [46]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [48]:
# model11 = get_conv3d_model_i()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 16, 60, 60, 16)    1312      
_________________________________________________________________
activation_29 (Activation)   (None, 16, 60, 60, 16)    0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 16, 60, 60, 16)    64        
_________________________________________________________________
max_pooling3d_26 (MaxPooling (None, 8, 30, 30, 16)     0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 8, 30, 30, 32)     13856     
_________________________________________________________________
activation_30 (Activation)   (None, 8, 30, 30, 32)     0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 8, 30, 30, 32)     128       
__________

In [49]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [50]:
# model11.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val Source path =  ; batch size = 40
Project_data/train ; batch size = 40
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/17 [==>...........................] - ETA: 44s - loss: 5.2984 - categorical_accuracy: 0.2000 

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 40s 2s/step - loss: 3.3401 - categorical_accuracy: 0.3594 - val_loss: 2.0814 - val_categorical_accuracy: 0.5000

Epoch 00001: saving model to model_init_2019-12-2222_33_48.524896/model-00001-3.39555-0.35445-2.08136-0.50000.h5
Epoch 2/50
17/17 [==============================] - 21s 1s/step - loss: 1.4316 - categorical_accuracy: 0.5166 - val_loss: 1.1717 - val_categorical_accuracy: 0.6333

Epoch 00002: saving model to model_init_2019-12-2222_33_48.524896/model-00002-1.43162-0.51662-1.17174-0.63333.h5
Epoch 3/50
17/17 [==============================] - 21s 1s/step - loss: 1.2323 - categorical_accuracy: 0.5700 - val_loss: 1.1938 - val_categorical_accuracy: 0.5667

Epoch 00003: saving model to model_init_2019-12-2222_33_48.524896/model-00003-1.24640-0.56676-1.19376-0.56667.h5
Epoch 4/50
17/17 [==============================] - 17s 992ms/step - loss: 0.9407 - categorical_accuracy: 0.6563 - val_loss: 0.9811 - val_categorical_accuracy: 0.5833

Epoch 000

17/17 [==============================] - 17s 973ms/step - loss: 0.1670 - categorical_accuracy: 0.9654 - val_loss: 0.6970 - val_categorical_accuracy: 0.8000

Epoch 00028: saving model to model_init_2019-12-2222_33_48.524896/model-00028-0.16699-0.96540-0.69698-0.80000.h5
Epoch 29/50
17/17 [==============================] - 16s 924ms/step - loss: 0.2150 - categorical_accuracy: 0.9377 - val_loss: 0.4993 - val_categorical_accuracy: 0.8167

Epoch 00029: saving model to model_init_2019-12-2222_33_48.524896/model-00029-0.21503-0.93772-0.49935-0.81667.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.
Epoch 30/50
17/17 [==============================] - 16s 941ms/step - loss: 0.1743 - categorical_accuracy: 0.9481 - val_loss: 0.5082 - val_categorical_accuracy: 0.8000

Epoch 00030: saving model to model_init_2019-12-2222_33_48.524896/model-00030-0.17432-0.94810-0.50818-0.80000.h5
Epoch 31/50
17/17 [==============================] - 16s 944ms/step - loss: 0.2066 - 

Epoch 23/50
17/17 [==============================] - 16s 945ms/step - loss: 0.1847 - categorical_accuracy: 0.9446 - val_loss: 0.3847 - val_categorical_accuracy: 0.9000

Epoch 00023: saving model to model_init_2019-12-2222_33_48.524896/model-00023-0.18473-0.94464-0.38475-0.90000.h5

## Model 12: Increasing the no of parameters by increasing image size to 100X100

In [59]:
# def get_conv3d_model_j():

# # Define model
#     model = Sequential()

#     model.add(Conv3D(16, kernel_size=(3,3,3), input_shape=input_shape,padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2,2,2)))

#     model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   

#     model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   
#     model.add(Conv3D(128, kernel_size=(3,3,3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling3D(pool_size=(2, 2, 2)))
   
#     #Flatten Layers
#     model.add(Flatten())
#     model.add(Dense(512, activation='relu'))
#     model.add(Dropout(0.60))

#     #softmax layer
#     model.add(Dense(5, activation='softmax'))


#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [60]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=100, numCols=100, numEpochs=50)

batch_size= 40 ,num_frames= 16 ,num_rows= 100 ,num_cols= 100 ,num_epochs= 50


In [61]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [62]:
# model12 = get_conv3d_model_j()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_41 (Conv3D)           (None, 16, 100, 100, 16)  1312      
_________________________________________________________________
activation_41 (Activation)   (None, 16, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization_41 (Batc (None, 16, 100, 100, 16)  64        
_________________________________________________________________
max_pooling3d_38 (MaxPooling (None, 8, 50, 50, 16)     0         
_________________________________________________________________
conv3d_42 (Conv3D)           (None, 8, 50, 50, 32)     13856     
_________________________________________________________________
activation_42 (Activation)   (None, 8, 50, 50, 32)     0         
_________________________________________________________________
batch_normalization_42 (Batc (None, 8, 50, 50, 32)     128       
__________

In [63]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [64]:
# model12.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/50


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 45s 3s/step - loss: 8.0675 - categorical_accuracy: 0.2989 - val_loss: 7.8148 - val_categorical_accuracy: 0.3800

Epoch 00001: saving model to model_init_2019-12-2222_33_48.524896/model-00001-8.11345-0.29563-7.81476-0.38000.h5
Epoch 2/50
17/17 [==============================] - 20s 1s/step - loss: 4.8926 - categorical_accuracy: 0.4041 - val_loss: 4.0570 - val_categorical_accuracy: 0.4833

Epoch 00002: saving model to model_init_2019-12-2222_33_48.524896/model-00002-4.89258-0.40409-4.05696-0.48333.h5
Epoch 3/50
17/17 [==============================] - 22s 1s/step - loss: 4.4898 - categorical_accuracy: 0.4633 - val_loss: 4.7583 - val_categorical_accuracy: 0.5333

Epoch 00003: saving model to model_init_2019-12-2222_33_48.524896/model-00003-4.50573-0.46049-4.75830-0.53333.h5
Epoch 4/50
17/17 [==============================] - 19s 1s/step - loss: 3.6875 - categorical_accuracy: 0.4489 - val_loss: 9.9204 - val_categorical_accuracy: 0.3000

Epoch 00004:

Epoch 40/50
17/17 [==============================] - 18s 1s/step - loss: 0.6211 - categorical_accuracy: 0.8235 - val_loss: 0.5359 - val_categorical_accuracy: 0.8333

Epoch 00040: saving model to model_init_2019-12-2222_33_48.524896/model-00040-0.62107-0.82353-0.53591-0.83333.h5

## Model 13 : CNN + LSTM Leveraging Transfer Learning

In [16]:
# from keras.applications import mobilenet
# base_model = mobilenet.MobileNet(input_shape=(128,128,3),weights='imagenet', include_top=False)

In [37]:
# def get_cnn_rnn_tl_model(gru_cells=64,dense_neurons=64,dropout=0.50):
    
#     # freeze the layers in base model
#     for layer in base_model.layers:
#         layer.trainable = False
        
#     model = Sequential()
#     model.add(TimeDistributed(base_model, input_shape=input_shape))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D((2, 2))))
#     model.add(TimeDistributed(Flatten()))              
    
#     model.add(GRU(gru_cells))
#     model.add(Dropout(dropout))
        
#     model.add(Dense(dense_neurons,activation='relu'))
#     model.add(Dropout(dropout))
    
#     model.add(Dense(5, activation='softmax'))
#     optimiser = optimizers.Adam()
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [41]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=120, numCols=120, numEpochs=25)

batch_size= 40 ,num_frames= 16 ,num_rows= 120 ,num_cols= 120 ,num_epochs= 25


In [42]:
# input_shape

(16, 120, 120, 3)

In [43]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [44]:
# model13 = get_cnn_rnn_tl_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_12 (TimeDis (None, 16, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_13 (TimeDis (None, 16, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_14 (TimeDis (None, 16, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 16, 1024)          0         
_________________________________________________________________
gru_3 (GRU)                  (None, 64)                209088    
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
__________

In [45]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [46]:
# model13.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 47s 3s/step - loss: 1.8003 - categorical_accuracy: 0.2708 - val_loss: 1.6047 - val_categorical_accuracy: 0.2600

Epoch 00001: saving model to model_init_2019-12-2303_29_20.233871/model-00001-1.80432-0.26998-1.60468-0.26000.h5
Epoch 2/25
17/17 [==============================] - 21s 1s/step - loss: 1.4966 - categorical_accuracy: 0.3478 - val_loss: 1.6014 - val_categorical_accuracy: 0.3333

Epoch 00002: saving model to model_init_2019-12-2303_29_20.233871/model-00002-1.49660-0.34783-1.60135-0.33333.h5
Epoch 3/25
17/17 [==============================] - 22s 1s/step - loss: 1.3304 - categorical_accuracy: 0.4289 - val_loss: 1.2947 - val_categorical_accuracy: 0.5167

Epoch 00003: saving model to model_init_2019-12-2303_29_20.233871/model-00003-1.33831-0.42507-1.29469-0.51667.h5
Epoch 4/25
17/17 [==============================] - 20s 1s/step - loss: 1.1578 - categorical_accuracy: 0.5449 - val_loss: 1.3918 - val_categorical_accuracy: 0.4000

Epoch 00004:

Epoch 18/25
17/17 [==============================] - 18s 1s/step - loss: 0.5825 - categorical_accuracy: 0.7958 - val_loss: 0.8253 - val_categorical_accuracy: 0.7000

Epoch 00018: saving model to model_init_2019-12-2303_29_20.233871/model-00018-0.58248-0.79585-0.82530-0.70000.h5

## Model 14 CNN+RNN Increase dense neurons to 256

In [61]:
# def get_cnn_rnn_tl_model_b(gru_cells=64,dense_neurons=256,dropout=0.50):
    
#     # freeze the layers in base model
#     for layer in base_model.layers:
#         layer.trainable = False
        
#     model = Sequential()
#     model.add(TimeDistributed(base_model, input_shape=input_shape))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D((2, 2))))
#     model.add(TimeDistributed(Flatten()))              
    
#     model.add(GRU(gru_cells))
#     model.add(Dropout(dropout))
        
#     model.add(Dense(dense_neurons,activation='relu'))
#     model.add(Dropout(dropout))
    
#     model.add(Dense(5, activation='softmax'))
#     optimiser = optimizers.Adam()
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [62]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=120, numCols=120, numEpochs=25)

batch_size= 40 ,num_frames= 16 ,num_rows= 120 ,num_cols= 120 ,num_epochs= 25


In [63]:
# input_shape

(16, 120, 120, 3)

In [64]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [65]:
# model14 = get_cnn_rnn_tl_model_b()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_25 (TimeDis (None, 16, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_26 (TimeDis (None, 16, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_27 (TimeDis (None, 16, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_28 (TimeDis (None, 16, 1024)          0         
_________________________________________________________________
gru_4 (GRU)                  (None, 64)                209088    
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               16640     
__________

In [66]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [67]:
# model14.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = Epoch 1/25
40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 46s 3s/step - loss: 1.7015 - categorical_accuracy: 0.2453 - val_loss: 1.4326 - val_categorical_accuracy: 0.4600

Epoch 00001: saving model to model_init_2019-12-2303_29_20.233871/model-00001-1.70085-0.24284-1.43259-0.46000.h5
Epoch 2/25
17/17 [==============================] - 22s 1s/step - loss: 1.3676 - categorical_accuracy: 0.4425 - val_loss: 1.2943 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2019-12-2303_29_20.233871/model-00002-1.36756-0.44246-1.29426-0.50000.h5
Epoch 3/25
17/17 [==============================] - 23s 1s/step - loss: 1.1294 - categorical_accuracy: 0.5628 - val_loss: 1.1481 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2019-12-2303_29_20.233871/model-00003-1.13730-0.55858-1.14806-0.50000.h5
Epoch 4/25
17/17 [==============================] - 20s 1s/step - loss: 0.8966 - categorical_accuracy: 0.6625 - val_loss: 1.1371 - val_categorical_accuracy: 0.5000

Epoch 00004:

Epoch 22/25
17/17 [==============================] - 18s 1s/step - loss: 0.2370 - categorical_accuracy: 0.9239 - val_loss: 0.7245 - val_categorical_accuracy: 0.7500

Epoch 00022: saving model to model_init_2019-12-2303_29_20.233871/model-00022-0.23703-0.92388-0.72450-0.75000.h5

## Model 15: Add drop out in the CNN layer and changed the learning rate to 0.001

In [17]:
# def get_cnn_rnn_tl_model_c(gru_cells=64,dense_neurons=256,dropout=0.50):
    
#     # freeze the layers in base model
#     for layer in base_model.layers:
#         layer.trainable = False
        
#     model = Sequential()
#     model.add(TimeDistributed(base_model, input_shape=input_shape))
#     model.add(TimeDistributed(BatchNormalization()))
#     model.add(TimeDistributed(MaxPooling2D((2, 2))))
#     model.add(Dropout(0.25))
    
#     model.add(TimeDistributed(Flatten()))              
    
#     model.add(GRU(gru_cells))
#     model.add(Dropout(dropout))
        
#     model.add(Dense(dense_neurons,activation='relu'))
#     model.add(Dropout(dropout))
    
#     model.add(Dense(5, activation='softmax'))
#     optimiser = optimizers.Adam(lr=0.001)
#     model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#     print (model.summary())
#     return model

In [18]:
# # Set Hyperparameters for the model
# set_hyper_parameters(batchSize=40, numFrames=16, numRows=120, numCols=120, numEpochs=25)

batch_size= 40 ,num_frames= 16 ,num_rows= 120 ,num_cols= 120 ,num_epochs= 25


In [19]:
# set_train_validation_steps()

steps_per_epoch= 17 validation_steps= 3


In [20]:
# model15 = get_cnn_rnn_tl_model_c()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 16, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 16, 1, 1, 1024)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 16, 1024)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                209088    
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
__________

In [21]:
# # create generators: Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.
# train_generator = generator(train_path, train_doc, batch_size)
# val_generator = generator(val_path, val_doc, batch_size)

In [22]:
# model15.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/25


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


17/17 [==============================] - 47s 3s/step - loss: 1.6811 - categorical_accuracy: 0.2530 - val_loss: 1.5390 - val_categorical_accuracy: 0.2700

Epoch 00001: saving model to model_init_2019-12-2304_27_59.294386/model-00001-1.68666-0.25490-1.53896-0.27000.h5
Epoch 2/25
17/17 [==============================] - 20s 1s/step - loss: 1.4532 - categorical_accuracy: 0.3939 - val_loss: 1.3939 - val_categorical_accuracy: 0.5333

Epoch 00002: saving model to model_init_2019-12-2304_27_59.294386/model-00002-1.45322-0.39386-1.39388-0.53333.h5
Epoch 3/25
17/17 [==============================] - 23s 1s/step - loss: 1.2277 - categorical_accuracy: 0.4715 - val_loss: 1.3898 - val_categorical_accuracy: 0.3667

Epoch 00003: saving model to model_init_2019-12-2304_27_59.294386/model-00003-1.22783-0.47139-1.38984-0.36667.h5
Epoch 4/25
17/17 [==============================] - 20s 1s/step - loss: 1.0883 - categorical_accuracy: 0.5511 - val_loss: 1.0757 - val_categorical_accuracy: 0.5667

Epoch 00004:

Epoch 7/25
17/17 [==============================] - 17s 1s/step - loss: 0.7119 - categorical_accuracy: 0.7405 - val_loss: 0.8585 - val_categorical_accuracy: 0.7000

Epoch 00007: saving model to model_init_2019-12-2304_27_59.294386/model-00007-0.71189-0.74048-0.85849-0.70000.h5